# Inspect forecast

In [ ]:
import pandas as pd
# from darksky import forecast
import requests
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Spectral6

output_notebook()   # so show() displays in the notebook

In [ ]:
path_list = ['33.3442655,126.4327353',
             '33.5266085,126.3729323',
             '33.4459325,126.9488463']
wind = []
for i, path_ in enumerate(path_list):
    response = requests.get('https://darksky.net/forecast/'+path_+'/ca12/en.json')
    df = pd.read_json(response.text)
    hourly_= df.hourly['data']
    wind.append(pd.DataFrame(hourly_).windSpeed.values)
#print(wind)

In [ ]:
# Plotting
s = figure(plot_width=800, plot_height=400, title=None)

x = range(1, len(wind[0])+1)
for i, wind_forecast in enumerate(wind):
    s.circle(x, wind_forecast, size=4, color=Spectral6[i], alpha=0.5, legend="%s" % path_list[i])

show(s)